# make Gini
# Use checkerpy
# Make logger
# Write dostrings
# Make sphinx docs
# Mkae python package.
# Discuss third algo!

In [1]:
import sys

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

In [3]:
sys.path.append('..')

In [4]:
from latenttopics import Corpus, Pipeline, Visualize
from latenttopics.pipeline import DEFAULT_PIPELINE
from latenttopics.algorithms import PLSA, ConditionalPLSA

[nltk_data] Downloading package stopwords to
[nltk_data]     ../latenttopics/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to ../latenttopics/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     ../latenttopics/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     ../latenttopics/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
data_dir = '../data'
file = 'Full-Economic-News-DFE-839861.csv'

options = {
    'encoding': 'latin_1'
}

In [ ]:
source = '/'.join([data_dir, file])

raw = pd.read_csv(source, **options)
raw.head()

In [ ]:
docs = raw.text.to_list()[:1000]
pipeline = Pipeline(*DEFAULT_PIPELINE)
corpus = Corpus(docs, pipeline)
corpus

In [ ]:
n_topics = 2

In [ ]:
conditional_plsa = ConditionalPLSA(corpus, n_topics)
conditional_result = conditional_plsa.fit()

In [ ]:
visualize = Visualize(conditional_result)

In [ ]:
plsa = PLSA(corpus, n_topics)
result = plsa.fit()

In [ ]:
visualize = Visualize(result)

In [ ]:
fig, ax = plt.subplots()
susi = visualize.convergence(ax)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
susi = visualize.topics(ax)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
susi = visualize.words_in_topic(0, ax)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
susi = visualize.words_in_topic(1, ax)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
susi = visualize.words_in_topic(2, ax)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
susi = visualize.topics_in_doc(2, ax)
fig.tight_layout()

In [ ]:
result.word_given_topic

In [ ]:
a.sum(axis=None)

In [ ]:
b = np.array([1])

In [ ]:
a/b

In [ ]:
plsa._PLSA__normalized(a, fail=True)

In [ ]:
plsa._PLSA__normalized(a, fail=False)

In [ ]:
raise NotImplementedError